## Impact of LRTSAC early bailout on precision and recall

File used to create figures 10 and 11 of the paper.

In [ ]:
import single_data_loader
%load_ext autoreload
%autoreload 2

In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

Glob path to the output files of the experiment:

In [ ]:
file_glob_format = "/home/riuclement/Documents/RANSAC-benchmark/ipol_experiment/qs_exp/*/experience_results/uniform/{}_{}_std{}_ratio{}_output_lrt.txt"

Options and default values of the experiment:

In [ ]:
algorithms = ["LRT all options", "LRT no bailout"]
num_metrics = 4
datasets = {"hom": 10, "fun": 11, "ess": 6}
sigma_values = np.arange(0, 3.1, 0.1)
ratio_values = np.arange(0, 1, 0.1)
num_runs = 25

Loading of the data:

In [ ]:
data_loader = single_data_loader.Metric_Data_Loader(
    file_glob_format,
    algorithms,
    num_metrics,
    datasets,
    sigma_values,
    ratio_values,
    num_runs
)

In [ ]:
data_loader.load_and_clean_data()

In [ ]:
def create_ticks(datasize):
    x_names = [
        "H{}".format(i) for i in range(1, datasize["hom"] + 1)
    ] + [
        "F{}".format(i) for i in range(1, datasize["fun"] + 1)
    ] + [
        "E{}".format(i) for i in range(1, datasize["ess"] + 1)
    ]
    x_ticks = [i + 1 for i in range(len(x_names))]
    return x_ticks, x_names

def plot_allmetrics_LRT(metric_index, metric_name, values, lim=False, save=False):
    x_ticks, x_names = create_ticks(values._datasets)
    metric_values = []
    for dataset_name in sorted(list(values._datasets.keys()), reverse=True):
        metric_values.append(values._mean_values[dataset_name][:, metric_index, :, :, :])
    metric_values = np.concatenate(metric_values, axis=1)
    metric_ratio = metric_values[0, :, :, :] / metric_values[1, :, :, :]
    metric_ratio[np.isnan(metric_ratio)] = np.nanmedian(metric_ratio)
    
    plt.figure(figsize=(10, 3))
    plt.title('{} of LRT with bailout over {} of LRT without bailout\nover all datasets and all settings'.format(metric_name, metric_name))
    plt.boxplot(metric_ratio.reshape(-1, metric_ratio.shape[1] * metric_ratio.shape[2]).T,
               sym="")
    plt.xlabel('Dataset names')
    plt.ylabel('{} ratio '.format(metric_name))
    plt.xticks(x_ticks, x_names)

    if lim:
        plt.ylim(0.9, 1.1)
    plt.tight_layout()
    if save:
        plt.savefig(save)
    else:
        plt.show()

Plot the precision and recall for LRTSac with and without early bailout:

In [ ]:
plot_allmetrics_LRT(0, 'Precision', data_loader, lim=False, save=False)
plot_allmetrics_LRT(1, 'Recall', data_loader, lim=True, save=False)